# Amex - trojans - Version 1

## Schedule
- feature engineering 1.5 months
- model tuning 1 month

This is a starter code template for catgorical variables. 

Mean, Max, Min, Last, Std and Count functions should always be considered for numerical variables.

Mean, Max and Min functions can not be implemented for cat variables

DDL: 6/22

Deliverables: A csv file/ pickle file

- Hongyi: Date + 'B_30', 'B_38', 

- Kate: 'D_114', 'D_116', 'D_117', 

- Duanmu: 'D_120', 'D_126', 'D_63',

- Ziyi: 'D_64', 'D_66', 'D_68'


In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import sklearn

In [2]:
# add data 
# Download: https://www.kaggle.com/datasets/munumbutt/amexfeather
# Or press "add data" and search "amexfeather", you will see AMEX-Feather-Dataset

train = pd.read_feather('../input/amexfeather/train_data.ftr')
test = pd.read_feather('../input/amexfeather/test_data.ftr')

In [3]:
tempTrain = train[['customer_ID','S_2','B_30','B_38','target']]
tempTrain.head()
del train
gc.collect()

In [4]:
tempTest = test[['customer_ID','S_2','B_30','B_38']]
tempTest.head()
del test
gc.collect()

In [5]:
tempTrain.info()

In [6]:
def dataTransform(df):
    df['S_2'] = pd.to_datetime(df['S_2'])
    df['Year'] = df['S_2'].apply(lambda x:x.year)
    df['Month'] = df['S_2'].apply(lambda x:x.month)
    df['Day'] = df['S_2'].apply(lambda x:x.day)
    df['Year_Month'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
#     df['Year_Month'] = df['Year_Month'].apply(lambda x: hash(x)%13)

dataTransform(tempTrain)

In [7]:
tempTrain.info()

In [8]:
tempTrain.head()

In [9]:
def plot_year_month(df,col):
    plt.figure(figsize=(12,6))
    data = df.groupby('Year_Month')[col].agg(['sum','count']).reset_index()
    print(data.columns)
    data['ratio'] = data['sum']/data['count']
    sns.barplot(x='Year_Month', y='ratio', data=data)

plot_year_month(tempTrain,'target')


In [10]:
def plot_B(df,col,name):
    plt.figure(figsize=(12,6))
    data = df.groupby(col)['target'].agg(['sum','mean']).reset_index()
    print(data.columns)
    sns.barplot(x=col, y=name, data=data)
#     sns.barplot(x=col, y='sum', data=data)

plot_B(tempTrain,'B_30','mean')
plot_B(tempTrain,'B_30','sum')
plot_B(tempTrain,'B_38','mean')
plot_B(tempTrain,'B_38','sum')

In [11]:
import random
from collections import Counter
def plot_time_feature(df):
    randCust = df[df['target']==1]['customer_ID'].unique()[random.randint(1,1000)]
    data = df[df['customer_ID']==randCust]
    values = [_+0.1 for _ in list(data['B_30'].astype('float32'))]
    print(values)
    print(Counter(data['B_30']))
    print('Target',data['target'].iloc[0])
    sns.histplot(x='B_30',data =data)
    sns.lineplot(data = values)

plot_time_feature(tempTrain)

In [12]:
plot_time_feature(tempTrain)

In [13]:
plot_time_feature(tempTrain)

In [14]:
plot_time_feature(tempTrain)

In [15]:
# b_30_feature(df,col):

# pct+sum 1,2 
def b30func1(df1):
    _ = list(df1)
    counter = Counter(_)
    return counter[1]/len(_)
def b30func2(df1):
    _ = list(df1)
    counter = Counter(_)
    return counter[2]/len(_)
def b30func3(df1):
    _ = list(df1)
    counter = Counter(_)
    return (counter[1]+counter[2])/len(_)
def b30func4(df1):
    _ = list(df1)
    return sum(_)

# volatility 1,2
# count of 0-1/2 pairs,即从0到1、2的转变有几次
def b30func5(df1):
    _ = list(df1)
    if len(_) == 1:
        return 0
    value = 0
    count = 0
    for i in range(0,len(_)):
        if (_[i] != 0) and (value==1) :
            count += 1
            value = 0
        elif _[i] == 0:
            value = 1
    return count/(len(_)-1)

# time_w_avg，根据之前的plot,0-1-2对target有影响，且影响的大小和0-1-2的大小关系相符，此处给予更靠后的记录的statement日更高的权重，线性加权
def b30func6(df):
    _ = list(df)
    weights = [(idx+1)/len(_)*value for idx,value in enumerate(_)]
    return sum(weights)

# 


dfb30 = tempTrain.groupby('customer_ID')['B_30'].agg([b30func1,b30func2,b30func3,b30func4,b30func5,b30func6])
dfb30_ = tempTest.groupby('customer_ID')['B_30'].agg([b30func1,b30func2,b30func3,b30func4,b30func5,b30func6])
dfb30.describe()

In [16]:
# b_38_feature(df,col):

# pct+sum 1,2,3| 4,5,6,7 
def b38func1(df1):
    _ = list(df1)
    counter = Counter(_)
    return (counter[1]+counter[2]+counter[3])/len(_)
def b38func3(df1):
    _ = list(df1)
    counter = Counter(_)
    return (counter[1]+counter[2])/len(_)
def b38func4(df1):
    _ = list(df1)
    counter = Counter(_)
    return (counter[4]+counter[6])/len(_)

# volatility 1,2
# count of 1,2,3 - 4,5,6,7 pairs,即从123到4567的转变有几次
def b38func5(df1):
    _ = list(df1)
    if len(_) == 1:
        return 0
    value = 0
    count = 0
    for i in range(0,len(_)):
        if (_[i] not in [1,2,3]) and (value==1) :
            count += 1
            value = 0
        elif _[i] in [1,2,3]:
            value = 1
    return count/(len(_)-1)

# time_w_avg，根据之前的plot，4567更趋向于导致违约，由于4567从图上也没有明显实际的对target的影响的大小关系，直接以4为thred，并且此处给予更靠后的记录的statement日更高的权重，线性加权
def b38func6(df):
    _ = list(df)
    weights = [(idx+1)/len(_)*(value>=4) for idx,value in enumerate(_)]
    return sum(weights)

# 


dfb38 = tempTrain.groupby('customer_ID')['B_38'].agg([b38func1,b38func3,b38func4,b38func5,b38func6])
dfb38_ = tempTest.groupby('customer_ID')['B_38'].agg([b38func1,b38func3,b38func4,b38func5,b38func6])

dfb38.describe()

In [17]:
# date feature

# registered date
def datefunc1(df):
    return (df.iloc[-1]-df.iloc[0]).days

tempTrain['S_2'] = pd.to_datetime(tempTrain['S_2'])
tempTest['S_2'] = pd.to_datetime(tempTest['S_2'])

dfdate = tempTrain.groupby('customer_ID')['S_2'].agg([datefunc1])
dfdate_ = tempTest.groupby('customer_ID')['S_2'].agg([datefunc1])

dfdate.describe()

In [18]:
# # month features

# # registered date
# def monthfunc1(df):
#     return 1*(df.iloc[-1]==3)

# dfmonth = tempTrain.groupby('customer_ID')['Month'].agg([monthfunc1])
# dfmonth.describe()

In [19]:
result = pd.concat([dfb30,dfb38,dfdate],axis=0)
result_ = pd.concat([dfb30_,dfb38_,dfdate_],axis=1)

In [20]:
result = pd.concat([dfb30,dfb38],axis=1)
result = pd.concat([result,dfdate],axis=1)
result.info()

In [21]:
result_ = pd.concat([dfb30_,dfb38_],axis=1)
result_ = pd.concat([result_,dfdate_],axis=1)
result_.info()

In [22]:
def reduce_size(df):
    for i in df.columns:
        df[i] = df[i].astype('float16')

reduce_size(result)
reduce_size(result_)

In [24]:
result.fillna(0,inplace = True)
result_.fillna(0,inplace=True)

In [25]:
result.to_pickle('shy_train.pkl')
result_.to_pickle('shy_test.pkl')